In [0]:
# 02_Silver_Governance (FIXED)

from pyspark.sql.functions import *
from pyspark.sql.types import *

# --- STEP 1: LOAD BRONZE ---
df_bronze_users = spark.table("bronze_users")
df_bronze_txns = spark.table("bronze_transactions")

# --- STEP 2: CLEANING ---
df_silver_users = df_bronze_users \
    .withColumn("Email", lower(col("Email"))) \
    .dropDuplicates(["CustomerID"])

# Clean Transactions
df_silver_txns = df_bronze_txns \
    .withColumn("Transaction_Date", to_timestamp(col("Transaction_Date"))) \
    .withColumn("Transaction_Amount", col("Transaction_Amount").cast("double")) \
    .fillna("Unknown", subset=["Anomaly_Reason"]) # Handle null reasons

# --- STEP 3: GOVERNANCE (Remove negative amounts) ---
clean_txns = df_silver_txns.filter(col("Transaction_Amount") >= 0)

# --- STEP 4: JOINING ---
df_joined = clean_txns.alias("t").join(
    df_silver_users.alias("u"),
    col("t.CustomerID") == col("u.CustomerID"),
    "left"
)

# --- STEP 5: SELECT FINAL COLUMNS (Matches YOUR Data Structure) ---
df_final = df_joined.select(
    col("t.TransactionID"),
    col("t.CustomerID"),
    col("u.First_Name"),
    col("u.Last_Name"),
    col("u.Age"),
    col("u.City").alias("User_City"),
    col("u.Experiment_Group"), 
    col("t.Transaction_Date"),
    col("t.Transaction_Amount"),
    col("t.Transaction_Type"),
    col("t.Merchant_City"),
    col("t.Merchant_Category"),      
    col("t.Transaction_Purpose"),    
    col("t.Device_Type"),            
    col("t.Anomaly"),
    col("t.Anomaly_Reason").alias("Anomaly_Type") # 
)

# --- STEP 6: SAVE & OPTIMIZE ---
df_final.write.format("delta").mode("overwrite").saveAsTable("silver_transactions")

spark.sql("OPTIMIZE silver_transactions ZORDER BY (Transaction_Date, CustomerID)")

print("✅ Silver Layer Updated Successfully.")
display(df_final.limit(5))

✅ Silver Layer Updated Successfully.


TransactionID,CustomerID,First_Name,Last_Name,Age,User_City,Experiment_Group,Transaction_Date,Transaction_Amount,Transaction_Type,Merchant_City,Merchant_Category,Transaction_Purpose,Device_Type,Anomaly,Anomaly_Type
TXN_1_0,1,Michelle,Hernandez,69,Cassandraton,Control,2023-08-13T18:45:26.082Z,13.02,Online,Cassandraton,Travel,Shopping,Mobile,0,None
TXN_1_1,1,Michelle,Hernandez,69,Cassandraton,Control,2023-08-14T05:28:48.880Z,115.83,Chip,Cassandraton,Healthcare,Healthcare,POS,0,None
TXN_1_2,1,Michelle,Hernandez,69,Cassandraton,Control,2023-08-14T18:15:49.300Z,3.04,Swipe,Cassandraton,Education,Travel,POS,0,None
TXN_1_3,1,Michelle,Hernandez,69,Cassandraton,Control,2023-08-14T21:05:28.222Z,58.65,Online,East Jessetown,Gas,Shopping,POS,0,None
TXN_1_4,1,Michelle,Hernandez,69,Cassandraton,Control,2023-08-15T18:29:40.980Z,199.96,Chip,Cassandraton,Healthcare,Entertainment,Mobile,0,None
